# Austin B-cycle Data Cleanup and Exploration

### Introduction 

This notebook contains the clean up and exploration for the publicly available data of Austin B-Cycle, a bike-sharing program. The notebook looks at the inconsistencies in the columns caused by missing data, duplicates, typos, and other anomalies. The result is a re-organized CSV file that has data ready for analysis.

### Data Extraction 

* The Austin B-Cycle data comes from the city of Austin Open Data Portal (AODP). The portal has API functionality through which we obtained the most current dataset. The API uses The Socrata Open Data API(SODA API) which hosts the AODP datasets. 

* As of early September 2018, there are a total of 991271 rows of rides. The dataset starts from the Austin B-Cycle inception in December 2013 and continues through July 2018.

* Data Provided 
    * Trip ID 
    * Membership Type 
    * Bicycle ID 
    * Checkout Time 
    * Checkout Kiosk ID
    * Checkout Kiosk
    * Return Kiosk ID
    * Trip Duration Minutes
    * Month
    * Year

##  Dependencies and API


* AODP Dataset Access: https://data.austintexas.gov/Transportation-and-Mobility/Austin-B-Cycle-Trips/tyfh-5r8s
* API Endpoint:  https://data.austintexas.gov/resource/cwi3-ckqi.json
* API Documentation: https://dev.socrata.com/foundry/data.austintexas.gov/cwi3-ckqi

To access the dataset host, install SODA API first: 

> pip install sodapy

The script works without a token and password, as Austin B-Cycle data is a public dataset. An individual API token and password may be created to avoid throttling limits. To create a token and password visit https://data.austintexas.gov/profile/app_tokens and specify the client information below on the corresponding commented section.

In [1]:
# Uncomment the command below if sodapy is not currently installed 
#!pip install sodapy

In [2]:
# Import Dependencies
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import csv
import requests
from pprint import pprint
from sodapy import Socrata

# Ignore Warnings as we are rewrititng values 
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.austintexas.gov", None)

# First 991271 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("cwi3-ckqi", limit= 991271)

# Convert to pandas DataFrame
df_bike= pd.DataFrame.from_records(results)

In [4]:
# Display the top rows of the dataframe 
df_bike.head()


,bicycle_id,checkout_date,checkout_kiosk,checkout_kiosk_id,checkout_time,membership_type,month,return_kiosk,return_kiosk_id,trip_duration_minutes,trip_id,year
0,207,2014-10-26T00:00:00.000,West & 6th St.,2537,13:12:00,Annual (San Antonio B-cycle),10,Rainey St @ Cummings,2707,76,9900285854,2014
1,969,2014-10-26T00:00:00.000,Convention Center / 4th St. @ MetroRail,2498,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Pfluger Bridge @ W 2nd Street,2566,58,9900285855,2014
2,214,2014-10-26T00:00:00.000,West & 6th St.,2537,13:12:00,Annual Membership (Austin B-cycle),10,8th & Congress,2496,8,9900285856,2014
3,745,2014-10-26T00:00:00.000,Zilker Park at Barton Springs & William Barton...,NaN,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Zilker Park at Barton Springs & William Barton...,NaN,28,9900285857,2014
4,164,2014-10-26T00:00:00.000,Bullock Museum @ Congress & MLK,2538,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Convention Center/ 3rd & Trinity,NaN,15,9900285858,2014


## Initial Data Exploration 

On an initial exploration of the data, we see that the columns month, year, membership type, bicycle id, return kiosk id, and check out kiosk id are missing values. It does not necessarily mean that all the data is not present, it just needs to be extracted and reformatted. The API returns all data types as strings or Pandas objects. Currently, there are 991271 rows of information and 12 columns. During the initial exploration we discovered we were missing values in the following columns:

* month 
* year 
* membership_type 
* bicycle id
* checkout_kiosk_id 
* return_kiosk_id  

In [5]:
# Count rows and columns 
df_bike.shape

(991271, 12)

In [6]:
# Check for missing values 
df_bike.count()

bicycle_id               990548
checkout_date            991271
checkout_kiosk           991271
checkout_kiosk_id        972240
checkout_time            991271
membership_type          984960
month                    618479
return_kiosk             991271
return_kiosk_id          971487
trip_duration_minutes    991271
trip_id                  991271
year                     618479
dtype: int64

In [7]:
# We see that the last values of our dataframe are missing the month and year
df_bike.tail()

,bicycle_id,checkout_date,checkout_kiosk,checkout_kiosk_id,checkout_time,membership_type,month,return_kiosk,return_kiosk_id,trip_duration_minutes,trip_id,year
991266,1524,2018-07-31T00:00:00.000,Capitol Station / Congress & 11th,2497,23:24:32,U.T. Student Membership,NaN,City Hall / Lavaca & 2nd,2499,29,18122309,NaN
991267,95,2018-07-31T00:00:00.000,Long Center @ South 1st & Riverside,2549,23:29:21,Local365+Guest Pass,NaN,East 6th & Pedernales St.,2544,18,18122312,NaN
991268,252,2018-07-31T00:00:00.000,Rio Grande & 28th,3793,23:39:16,U.T. Student Membership,NaN,22nd & Pearl,3792,4,18122330,NaN
991269,2228,2018-07-31T00:00:00.000,Dean Keeton & Whitis,3795,23:42:50,U.T. Student Membership,NaN,Nueces & 26th,3838,2,18122340,NaN
991270,576,2018-07-31T00:00:00.000,Rio Grande & 28th,3793,23:44:32,U.T. Student Membership,NaN,Rio Grande & 28th,3793,4,18122349,NaN


In [8]:
# Count how many rows are missing year, month, membership_type, bicycle_id 
missing_year = df_bike["year"].isnull().sum()
missing_month = df_bike["month"].isnull().sum()
missing_membership_type = df_bike["membership_type"].isnull().sum()
missing_bike_id = df_bike["bicycle_id"].isnull().sum()

# Create summary table of the missing values 
print(f"There are {missing_year} missing year values.")
print(f"There are {missing_month} missing month values.")
print(f"There are {missing_membership_type} missing membership type values.")
print(f"There are {missing_bike_id} missing bike id values.")

There are 372792 missing year values.
There are 372792 missing month values.
There are 6311 missing membership type values.
There are 723 missing bike id values.


In [9]:
df_bike.dtypes

bicycle_id               object
checkout_date            object
checkout_kiosk           object
checkout_kiosk_id        object
checkout_time            object
membership_type          object
month                    object
return_kiosk             object
return_kiosk_id          object
trip_duration_minutes    object
trip_id                  object
year                     object
dtype: object

## Initial Data Clean Up

In this section, we address the missing values found in the initial exploration.

* First, we use the check_out date to extract the year, month, trip date, and trip day of the week. This is accomplished through the use of the datetime function.
Next, we split the hour from the checkout time. This is accomplished with an anonymous function.
Also, we change the data types of trip duration from object to integer type. After this change all the data types are ready for iteration in the analysis notebook.
Finally, we rename the columns to further explore and understand the remaining columns of missing and conflicting values: 

    * membership_type
    * checkout_kiosk_id
    * return_kiosk_id


In [10]:
# Renaming the data frame
df_bike_clean = df_bike

In [11]:
# Checking whether the checkout date matches with the number of trips (Trip ID) taken  
df_bike_clean.count()

bicycle_id               990548
checkout_date            991271
checkout_kiosk           991271
checkout_kiosk_id        972240
checkout_time            991271
membership_type          984960
month                    618479
return_kiosk             991271
return_kiosk_id          971487
trip_duration_minutes    991271
trip_id                  991271
year                     618479
dtype: int64

In [12]:
# We see take the Checkout Date and extract the Month, Year, and Day of the Week 
df_bike_clean['checkout_date'] = pd.to_datetime(df_bike_clean['checkout_date']) 
df_bike_clean['year'] = df_bike_clean['checkout_date'].dt.year
df_bike_clean['month'] = df_bike_clean['checkout_date'].dt.month
df_bike_clean['Trip Date'] = df_bike_clean['checkout_date'].dt.day
df_bike_clean['Trip Day of Week'] = df_bike_clean['checkout_date'].dt.weekday_name

In [13]:
# Inspect the filled in values 
df_bike_clean.tail()

,bicycle_id,checkout_date,checkout_kiosk,checkout_kiosk_id,checkout_time,membership_type,month,return_kiosk,return_kiosk_id,trip_duration_minutes,trip_id,year,Trip Date,Trip Day of Week
991266,1524,2018-07-31,Capitol Station / Congress & 11th,2497,23:24:32,U.T. Student Membership,7,City Hall / Lavaca & 2nd,2499,29,18122309,2018,31,Tuesday
991267,95,2018-07-31,Long Center @ South 1st & Riverside,2549,23:29:21,Local365+Guest Pass,7,East 6th & Pedernales St.,2544,18,18122312,2018,31,Tuesday
991268,252,2018-07-31,Rio Grande & 28th,3793,23:39:16,U.T. Student Membership,7,22nd & Pearl,3792,4,18122330,2018,31,Tuesday
991269,2228,2018-07-31,Dean Keeton & Whitis,3795,23:42:50,U.T. Student Membership,7,Nueces & 26th,3838,2,18122340,2018,31,Tuesday
991270,576,2018-07-31,Rio Grande & 28th,3793,23:44:32,U.T. Student Membership,7,Rio Grande & 28th,3793,4,18122349,2018,31,Tuesday


In [14]:
# Split the hour from the checkout time
df_bike_clean['Trip Hour'] = df_bike_clean['checkout_time'].apply(lambda x: x.split(":")[0])


In [15]:
# Inspect the filled in values 
df_bike_clean.head()

,bicycle_id,checkout_date,checkout_kiosk,checkout_kiosk_id,checkout_time,membership_type,month,return_kiosk,return_kiosk_id,trip_duration_minutes,trip_id,year,Trip Date,Trip Day of Week,Trip Hour
0,207,2014-10-26,West & 6th St.,2537,13:12:00,Annual (San Antonio B-cycle),10,Rainey St @ Cummings,2707,76,9900285854,2014,26,Sunday,13
1,969,2014-10-26,Convention Center / 4th St. @ MetroRail,2498,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Pfluger Bridge @ W 2nd Street,2566,58,9900285855,2014,26,Sunday,13
2,214,2014-10-26,West & 6th St.,2537,13:12:00,Annual Membership (Austin B-cycle),10,8th & Congress,2496,8,9900285856,2014,26,Sunday,13
3,745,2014-10-26,Zilker Park at Barton Springs & William Barton...,NaN,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Zilker Park at Barton Springs & William Barton...,NaN,28,9900285857,2014,26,Sunday,13
4,164,2014-10-26,Bullock Museum @ Congress & MLK,2538,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Convention Center/ 3rd & Trinity,NaN,15,9900285858,2014,26,Sunday,13


In [16]:
# Check for missing values 
df_bike_clean.count()

bicycle_id               990548
checkout_date            991271
checkout_kiosk           991271
checkout_kiosk_id        972240
checkout_time            991271
membership_type          984960
month                    991271
return_kiosk             991271
return_kiosk_id          971487
trip_duration_minutes    991271
trip_id                  991271
year                     991271
Trip Date                991271
Trip Day of Week         991271
Trip Hour                991271
dtype: int64

In [17]:
# Verify Data types
df_bike_clean.dtypes

bicycle_id                       object
checkout_date            datetime64[ns]
checkout_kiosk                   object
checkout_kiosk_id                object
checkout_time                    object
membership_type                  object
month                             int64
return_kiosk                     object
return_kiosk_id                  object
trip_duration_minutes            object
trip_id                          object
year                              int64
Trip Date                         int64
Trip Day of Week                 object
Trip Hour                        object
dtype: object

In [18]:
# Convert trip duration minutes from object to integer
df_bike_change1 = df_bike_clean["trip_duration_minutes"].astype(int)
df_bike_clean["trip_duration_minutes"] = df_bike_change1



In [19]:
# Convert bicycle ID from object to integer
df_bike_change2 = df_bike_clean["bicycle_id"].astype(float)
df_bike_clean["bicycle_id"] = df_bike_change2

In [20]:
df_bike_clean.dtypes

bicycle_id                      float64
checkout_date            datetime64[ns]
checkout_kiosk                   object
checkout_kiosk_id                object
checkout_time                    object
membership_type                  object
month                             int64
return_kiosk                     object
return_kiosk_id                  object
trip_duration_minutes             int32
trip_id                          object
year                              int64
Trip Date                         int64
Trip Day of Week                 object
Trip Hour                        object
dtype: object

In [21]:
# Rename the Columns' Name

df_bike_clean = df_bike_clean.rename(columns = {
    "bicycle_id": "Bicycle ID",
    "checkout_date": "Checkout Date",
    "checkout_kiosk": "Checkout Station",
    "checkout_kiosk_id": "Checkout Station ID",
    "checkout_time": "Checkout Time",
    "membership_type": "Membership Type",
    "month": "Trip Month",
    "return_kiosk":"Return Station",
    "return_kiosk_id": "Return Station ID",
    "trip_duration_minutes": "Trip Duration Minutes",
    "trip_id": "Trip ID",
    "year":"Trip Year",
    "Checkout Kiosk ID":"Checkout Station ID"
                                         })

df_bike_clean.head(1) 

,Bicycle ID,Checkout Date,Checkout Station,Checkout Station ID,Checkout Time,Membership Type,Trip Month,Return Station,Return Station ID,Trip Duration Minutes,Trip ID,Trip Year,Trip Date,Trip Day of Week,Trip Hour
0,207.0,2014-10-26,West & 6th St.,2537,13:12:00,Annual (San Antonio B-cycle),10,Rainey St @ Cummings,2707,76,9900285854,2014,26,Sunday,13


##  Station ID and Station Name Exploration and Cleanup

There are a significant number of values missing in the Checkout and Return Station ID columns. However, all the station names are available. In this section, we seek to understand and quantify the remaining missing values. 

* The missing values are summarized below. The only column with missing data that will be used in the analysis notebook is the membership_type. Thus, we need to fill-in the remaining missing values with zeros and explain the missing data. To accomplish this we fill missing values with zero.
* Besides missing values, we find that there are cases where there are more than one value assigned to a key in the Station ID columns.
* Ultimately, we make a new data frame that reorganizes the column order and focuses on the columns we will iterate through in the analysis section. 

In [22]:
# To check how many Checkout Station ID are blank
number_df_bike_checkout_id_blank  = df_bike_clean["Checkout Station ID"].isnull().sum()
print(f"There are {number_df_bike_checkout_id_blank} Checkout Station IDs that are blank.")

There are 19031 Checkout Station IDs that are blank.


In [23]:
# Fill the Na values with zero for exploration
df_bike_na = df_bike_clean.fillna(0)

In [24]:
# To find which check out station have blank checkout IDs that is zero or #N/A
df_bike_checkout_id_blank = df_bike_na.loc[(df_bike_na["Checkout Station ID"] == 0) | (df_bike_na["Checkout Station ID"] == "#N/A")]
df_bike_checkout_id_blank["Checkout Station"].value_counts()

Zilker Park at Barton Springs & William Barton Drive    11534
Dean Keeton & Speedway                                   3825
ACC - West & 12th                                        2462
Convention Center/ 3rd & Trinity                         1292
Mobile Station                                           1183
East 11th Street at Victory Grill                        1030
Red River @ LBJ Library                                   584
Mobile Station @ Bike Fest                                516
Main Office                                               300
Bullock Museum @ Congress & MLK                           172
State Capitol @ 14th & Colorado                           111
MapJam at Pan Am Park                                      32
MapJam at French Legation                                  27
MapJam at Hops & Grain Brewery                             19
Repair Shop                                                15
MapJam at Scoot Inn                                        11
Shop    

In [25]:
# List of checkout station id
checkout_station_id_list =  df_bike_na["Checkout Station ID"].value_counts().index
checkout_station_id_list

# Currently we have 83 unique check out station ID excluding zero and #N/A

Index(['3798', '2575', '2499', '2494', '2501', '2707', '2495', '2498', '2563',
       '2497', '2566', '2552', '2548', '2549', '2567', '2574', '2711', '2502',
            0, '2503', '2547', '2570', '2539', '2572', '2496', '2504', '3841',
       '2537', '3792', '2542', '3377', '2565', '3390', '2571', '2538', '3793',
       '3838', '2550', '2569', '3794', '2562', '3795', '3513', '2540', '3797',
       '2822', '2564', '3619', '3621', '2561', '3799', '2536', '3455', '2544',
       '3292', '2568', '2541', '3687', '1007', '1008', '3291', '3684', '3293',
       '#N/A', '3686', '3660', '2712', '2823', '2576', '3294', '3685', '2546',
       '2545', '3635', '1006', '1002', '3464', '3790', '1003', '3381', '2500',
       '3791', '3456', '1005', '1001'],
      dtype='object')

In [26]:
# Number of stations which have no checkout ID
blank_stations = len(df_bike_checkout_id_blank["Checkout Station"].value_counts())

# Number of unique checkout stations
unique_checkout_stations = df_bike_clean["Checkout Station"].unique().size

# Number of stations with unique checkout station ids other than zero
unique_stations_nonnull = unique_checkout_stations - blank_stations 

# Number of unique checkout station ID
unique_checkout_id = df_bike_clean["Checkout Station ID"].unique().size
unique_checkout_id_not_zero = unique_checkout_id - 2
# -2 for zero and #N/A checkout id

# Summary checkout station ID findings 
print(f"There are {blank_stations} stations without checkout ID.")
print(f"There are {unique_checkout_stations} Unique Checkout Station.")
print(f"There are {unique_stations_nonnull} stations with unique checkout station ids other than zero and #N/A.")
print(f"There are {unique_checkout_id_not_zero} Unique Checkout Station ID.")

There are 24 stations without checkout ID.
There are 104 Unique Checkout Station.
There are 80 stations with unique checkout station ids other than zero and #N/A.
There are 83 Unique Checkout Station ID.


In [27]:
# We have a larger number of unique checkout station ids than number of unique checkout stations. 
# This implies we have a few checkout stations with more than one checkout station IDs.

# Create a dictionary using keyword arguments: checkout station and checkout station ids 
Checkout_station_id = dict()
for index, row in df_bike_na.iterrows():
    if row['Checkout Station'] not in Checkout_station_id:
        Checkout_station_id[row['Checkout Station']] = set()
    else:
         Checkout_station_id[row['Checkout Station']].add(row['Checkout Station ID'])

In [28]:
# To check which station has more than one checkout id
Checkout_id_check = Checkout_station_id
for key in Checkout_id_check:
    if len(Checkout_id_check[key]) > 1:
        print("{} has both ids: {}".format(key, Checkout_id_check[key]))

Bullock Museum @ Congress & MLK has both ids: {'2538', '#N/A'}
State Capitol @ 14th & Colorado has both ids: {'2541', '#N/A'}
Main Office has both ids: {0, '1001', '#N/A'}
Lavaca & 6th has both ids: {'1007', '3294'}
Re-branding has both ids: {0, '#N/A'}
Repair Shop has both ids: {0, '#N/A'}
Republic Square @ 5th & Guadalupe has both ids: {'3456', '3455'}
Dean Keeton & Speedway  has both ids: {'3794', '#N/A'}


In [29]:
# To check if there is an overlap of Checkout Station ID :

# Remove station with zero checkout station ID 
for key in Checkout_id_check:
    if 0 in Checkout_id_check[key]:
        Checkout_id_check[key].remove(0)

# Remove station with "#N/A" checkout station ID 
for key in Checkout_id_check:
    if "#N/A" in Checkout_id_check[key]:
        Checkout_id_check[key].remove("#N/A")
        
# Check for an overlap of station IDs
for key1 in Checkout_id_check:
    for key2 in Checkout_id_check:
        if key1 == key2:
            continue
        intersect = Checkout_id_check[key1].intersection(Checkout_id_check[key2])
        if len(intersect) > 0:
            print("{} and {} share id: {}".format(key1, key2, intersect))

Lavaca & 6th and Guadalupe & 6th share id: {'3294'}
Republic Square @ Federal Courthouse Plaza and Republic Square @ 5th & Guadalupe share id: {'3455'}
Guadalupe & 6th and Lavaca & 6th share id: {'3294'}
Republic Square @ 5th & Guadalupe and Republic Square @ Federal Courthouse Plaza share id: {'3455'}


## Cleaning the Trip Duration and Bike ID Columns 

1.	 Looking closely into the data we found that some trip duration values are unusually large. During investigation, we found that under the Return Station column, those bikes were labeled as either stolen or missing. We are dropping the rows for our final clean CSV file for those missing and stolen bikes.

2.	 We also found that some bikes have a trip duration of zero minutes. This data is correct. Some of this can be explained by the bikes being transported by the company to different stations and marked as zero minutes. (https://austinstartups.com/analyzing-austin-bcycle-rentals-stations-d9a1863d17e9) 

3. We are also dropping the missing bicycle IDs. They do not provide us with relevant data.

In [30]:
# To check how many bikes were stolen
# These bikes have unusally large trip durations
df_bike_stolen = df_bike_clean.loc[df_bike_clean["Return Station"] == "Stolen"]
number_bike_stolen = df_bike_stolen["Return Station"].count()


# To check how many bikes were missing
# These bikes have unusally large trip durations
df_bike_missing = df_bike_clean.loc[df_bike_clean["Return Station"] == "Missing"]
number_bike_missing = df_bike_missing["Return Station"].count()


# To check how many bikes have trip duration with zero minutes
df_bike_trip_minutes_zero = df_bike_clean.loc[df_bike_clean["Trip Duration Minutes"] == 0]
number_bike_trip_minutes_zero = df_bike_trip_minutes_zero["Trip ID"].count()

# Summary of bike trips with distinct data 
print(f"There are {number_bike_stolen} bikes reported as stolen.")
print(f"There are {number_bike_missing} bikes reported as missing.")
print(f"There are {number_bike_trip_minutes_zero} bikes that had a trip duration of zero.")

There are 23 bikes reported as stolen.
There are 25 bikes reported as missing.
There are 19033 bikes that had a trip duration of zero.


In [31]:
# Drop the Rows for Stolen and Missing Bikes
# Create a clean data frame that does not include missing and stolen bikes
df_bike_clean = df_bike_clean.loc[(df_bike_clean["Return Station"] != "Stolen") & (df_bike_clean["Return Station"] != "Missing"), :]

In [32]:
# Check the data frame size after dropping the rows
df_bike_clean.shape

(991223, 15)

In [33]:
# Display the clean data frame
df_bike_clean.head()

,Bicycle ID,Checkout Date,Checkout Station,Checkout Station ID,Checkout Time,Membership Type,Trip Month,Return Station,Return Station ID,Trip Duration Minutes,Trip ID,Trip Year,Trip Date,Trip Day of Week,Trip Hour
0,207.0,2014-10-26,West & 6th St.,2537,13:12:00,Annual (San Antonio B-cycle),10,Rainey St @ Cummings,2707,76,9900285854,2014,26,Sunday,13
1,969.0,2014-10-26,Convention Center / 4th St. @ MetroRail,2498,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Pfluger Bridge @ W 2nd Street,2566,58,9900285855,2014,26,Sunday,13
2,214.0,2014-10-26,West & 6th St.,2537,13:12:00,Annual Membership (Austin B-cycle),10,8th & Congress,2496,8,9900285856,2014,26,Sunday,13
3,745.0,2014-10-26,Zilker Park at Barton Springs & William Barton...,NaN,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Zilker Park at Barton Springs & William Barton...,NaN,28,9900285857,2014,26,Sunday,13
4,164.0,2014-10-26,Bullock Museum @ Congress & MLK,2538,13:12:00,24-Hour Kiosk (Austin B-cycle),10,Convention Center/ 3rd & Trinity,NaN,15,9900285858,2014,26,Sunday,13


In [34]:
# Clean the Missing Bicycle ID’s
# We are dropping the rows for our final clean CSV file for the unlabeled bikes.
# Filling the Na values with zero for exploration
df_bike_na = df_bike_clean.fillna(0)

In [35]:
# Create a clean data frame that does not include unlabeled bikes
df_bike_clean = df_bike_na.loc[( df_bike_na["Bicycle ID"] != 0), :]

In [36]:
# Check the data frame size after dropping the rows
df_bike_clean.shape

(990500, 15)

## The Reorganized and Updated Data Frame

After the initial exploration of the data and clean up, we have a better grasp of the data. The columns that were missing significant data and had abnormalities are accounted for.

In the end, we won't need the following columns: bike id, checkout station id, return station ids. For our analysis, we now take only a subset of the most complete data.

In [37]:
# Organize column and take columns with relevant data which we will use for analysis
df_bike_clean = df_bike_clean[["Trip ID", "Membership Type", "Checkout Date","Checkout Time", "Checkout Station", "Return Station", "Trip Duration Minutes", "Trip Month", "Trip Year","Trip Date", "Trip Day of Week", "Trip Hour"]] 

## Cleaning and Grouping Membership Data 

Even though the data count reflects a complete dataset now, there is further cleaning that needs to happen before we can successfully analyze the data.

In this last part, we want to take a closer look to understand the types of memberships. Upon closer inspection, there are memberships with similar names that should be categorized together. For example: U.T. Student Membership and UT Student Membership.

We also find that there are two memberships RESTRICTED and PROHIBITED that yield trip duration average of zero minutes. We will drop this small data subset (18 rows) to categorize and analyze correctly. 

Furthermore, it will be more helpful to categorize the data by day, weekend, week, month, year, 3 year, and U.T. student memberships.


In [38]:
df_bike_clean["Membership Type"].value_counts()

Walk Up                                          368029
Local365                                         167270
U.T. Student Membership                          158477
24-Hour Kiosk (Austin B-cycle)                   108466
Local30                                           54729
Weekender                                         43860
Annual Membership (Austin B-cycle)                30243
Explorer                                          14853
Local365+Guest Pass                               10331
0                                                  6311
Local365 ($80 plus tax)                            4004
Founding Member                                    3546
7-Day                                              3124
Founding Member (Austin B-cycle)                   2763
7-Day Membership (Austin B-cycle)                  2752
Semester Membership (Austin B-cycle)               2418
Annual                                             1085
Semester Membership                             

In [39]:
# Examine Prohibited Memberships
test = df_bike_clean.loc[df_bike_clean["Membership Type"] == "PROHIBITED", :]
test.head()

,Trip ID,Membership Type,Checkout Date,Checkout Time,Checkout Station,Return Station,Trip Duration Minutes,Trip Month,Trip Year,Trip Date,Trip Day of Week,Trip Hour
326220,8482437,PROHIBITED,2016-01-20,15:34:07,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,20,Wednesday,15
329487,8370751,PROHIBITED,2016-01-11,14:14:10,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,11,Monday,14
329488,8370756,PROHIBITED,2016-01-11,14:14:46,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,1,1,2016,11,Monday,14
329489,8370766,PROHIBITED,2016-01-11,14:15:09,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,11,Monday,14
347380,9900014475,PROHIBITED,2016-01-11,14:12:10,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,11,Monday,14


In [40]:
# Examine Prohibited Memberships
test = df_bike_clean.loc[df_bike_clean["Membership Type"] == "RESTRICTED", :]
test.head()

,Trip ID,Membership Type,Checkout Date,Checkout Time,Checkout Station,Return Station,Trip Duration Minutes,Trip Month,Trip Year,Trip Date,Trip Day of Week,Trip Hour
326221,8482474,RESTRICTED,2016-01-20,15:35:09,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,20,Wednesday,15
329490,8368765,RESTRICTED,2016-01-11,11:30:08,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,8,1,2016,11,Monday,11
329491,8370726,RESTRICTED,2016-01-11,14:12:09,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,1,1,2016,11,Monday,14
329492,8370745,RESTRICTED,2016-01-11,14:13:32,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,0,1,2016,11,Monday,14
329493,8370747,RESTRICTED,2016-01-11,14:13:50,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,1,1,2016,11,Monday,14


In [41]:
# Replace all 24-hour with same name == day 
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {"24-Hour Kiosk (Austin B-cycle)": "day",
     "24-Hour-Online (Austin B-cycle)": "day",
     "24-Hour Membership (Austin B-cycle)": "day",
    "Explorer": "day", 
    "Explorer ($8 plus tax)":"day",
    "Try Before You Buy Special": "day",
    "RideScout Single Ride": "day", 
    "Aluminum Access":"day"})

# Replace all weekend membership == walk-up 
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {"Walk Up": "walk up", 0 :"walk up"})


# Replace all weekend membership == weekend 
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "Weekender": "weekend", 
        "Weekender ($15 plus tax)": "weekend", 
        "ACL Weekend Pass Special (Austin B-cycle)": "weekend", 
        "FunFunFun Fest 3 Day Pass": "weekend"
    })

# Replace all weekend membership == week
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "7-Day": "week", 
        "7-Day Membership (Austin B-cycle)": "week", 
    })


# Replace all weekend membership == month
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "Local30": "month", 
        "Local30 ($11 plus tax)": "month",
        "Madtown Monthly":"month", 
    })


# Combine all student memberships
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "U.T. Student Membership": "student",
        "UT Student Membership": "student", 
        "Semester Membership (Austin B-cycle)":"student", 
        "Semester Membership": "student"
    })

# Replace all annual membership == year
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "Annual Membership (Austin B-cycle)": "year",
         "Annual Member": "year",
         "Annual Membership":"year",
         "Annual (San Antonio B-cycle)": "year",
         "Annual Member (Houston B-cycle)":"year",
         "Annual Membership (Fort Worth Bike Sharing)":"year",
         "Annual (Denver B-cycle)":"year",
         "Republic Rider (Annual)":"year",
         "Republic Rider": "year",
         "Annual Plus":"year",
         "Annual (Madison B-cycle)":"year",
         "Annual (Broward B-cycle)":"year",
         "Annual (Denver Bike Sharing)":"year",
         "Annual (Boulder B-cycle)":"year",
         "Annual Membership (GREENbike)":"year",
         "Annual Pass":"year",
         "Annual (Kansas City B-cycle)":"year",
         "Annual (Cincy Red Bike)":"year",
         "Annual (Nashville B-cycle)":"year",
         "Annual Plus Membership":"year",
         "Annual Membership (Charlotte B-cycle)":"year",
         "Annual Membership (Indy - Pacers Bikeshare )":"year",
         "Annual (Omaha B-cycle)":"year",
         "Annual":"year",
         "Annual ": "year",
         "Local365": "year", 
         "Local365+Guest Pass":"year",
         "Local365 ($80 plus tax)": "year",
         "Local365 Youth with helmet (age 13-17 riders)": "year", 
         "Local365 Youth (age 13-17 riders)":"year",
         "Membership: pay once  one-year commitment":"year"
        
    })

# Replace all founding membership == 3 year
df_bike_clean["Membership Type"] = df_bike_clean["Membership Type"].replace(
    {
        "Founding Member": "3 year",
        "Founding Member (Austin B-cycle)": "3 year",
        "Denver B-cycle Founder": "3 year"
    })


In [42]:
# Create a new data frame that does not include restricted and prohibited
df_bike_clean = df_bike_clean.loc[(df_bike_clean["Membership Type"] != "RESTRICTED") & (df_bike_clean["Membership Type"] != "PROHIBITED"), :]

In [43]:
df_bike_clean.shape

(990482, 12)

In [44]:
# Verify clean up 
df_bike_clean["Membership Type"].value_counts()

walk up    374340
year       216566
student    161804
day        125192
month       55605
weekend     44780
3 year       6319
week         5876
Name: Membership Type, dtype: int64

In [45]:
# Fill the na values  
df_bike_clean = df_bike_clean.fillna(0)
df_bike_clean.count()

Trip ID                  990482
Membership Type          990482
Checkout Date            990482
Checkout Time            990482
Checkout Station         990482
Return Station           990482
Trip Duration Minutes    990482
Trip Month               990482
Trip Year                990482
Trip Date                990482
Trip Day of Week         990482
Trip Hour                990482
dtype: int64

In [46]:
# Export to csv
df_bike_clean.to_csv("Clean_Data\out.csv", index = None)